<a href="https://colab.research.google.com/github/lbhagavan/stanford_LLM_Leela/blob/homework/Tech16_final_av.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Document Summarizer supporting audio, video, pdf, docx, txt, and yourube video link

In [ ]:
# -*- coding: utf-8 -*-
"""Video and Audio Summarizer for Colab

This script allows users to summarize YouTube videos, uploaded video files, or audio files.
"""

'Video and Audio Summarizer for Colab\n\nThis script allows users to summarize YouTube videos, uploaded video files, or audio files.\n'

In [ ]:
# Install necessary packages
!pip install -q yt-dlp pydub llama-index-readers-file ipywidgets openai PyPDF2 python-docx
!pip install -q git+https://github.com/openai/whisper.git

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 170.1/170.1 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 50.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 362.9/362.9 kB 22.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 18.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 21.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 42.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.8/295.8 kB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 157.2/157.2 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 73.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1

In [ ]:
import yt_dlp
import os
import subprocess
from pathlib import Path
from google.colab import files
from llama_index.readers.file import VideoAudioReader
from openai import OpenAI
import ipywidgets as widgets
from IPython.display import display
import textwrap
import mimetypes
import shutil
import PyPDF2
from docx import Document

In [ ]:
# Global variable to track processed files
processed_files = set()

In [ ]:
# Set up OpenAI API key
from google.colab import userdata
open_ai_key = userdata.get('OPENAI_API_KEY')
os.environ["OPENAI_API_KEY"] = open_ai_key
client = OpenAI(api_key=open_ai_key)

In [ ]:
def download_audio(video_url, output_path='audio.wav'):
    ydl_opts = {
        'format': 'bestaudio/best',
        'outtmpl': output_path,
        'postprocessors': [{
            'key': 'FFmpegExtractAudio',
            'preferredcodec': 'wav',
            'preferredquality': '192',
        }],
    }
    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        ydl.download([video_url])

    # Check if the file was renamed
    if os.path.exists(output_path + '.wav'):
        os.rename(output_path + '.wav', output_path)

    print(f"Debug: Audio file downloaded as {output_path}")
    return output_path

In [ ]:
def process_audio(file_path):
    print(f"Debug: Processing audio file: {file_path}")
    try:
        reader = VideoAudioReader()
        documents = reader.load_data(Path(file_path))
        transcript = documents[0].text
        print("Transcript:")
        print(transcript)

        summary = summarize_text(transcript)
        print("\nAudio/Video Summary:")
        print(textwrap.fill(summary, width=80))
    except Exception as e:
        print(f"Error processing audio: {str(e)}")
        # If there's an error with VideoAudioReader, try using whisper directly
        try:
            import whisper
            model = whisper.load_model("base")
            result = model.transcribe(file_path)
            transcript = result["text"]
            print("Transcript (using whisper directly):")
            print(transcript)

            summary = summarize_text(transcript)
            print("\nAudio/Video Summary:")
            print(textwrap.fill(summary, width=80))
        except Exception as e:
            print(f"Error processing audio with whisper: {str(e)}")

In [ ]:
def determine_input_type(url, uploaded_file):
    if url:
        return 'youtube'
    elif uploaded_file:
        file_name = uploaded_file.get('name', '')
        print(f"Debug: File name in determine_input_type: {file_name}")

        if not file_name:
            return 'unknown'

        # Check file extension
        file_extension = os.path.splitext(file_name)[1].lower()
        print(f"Debug: File extension is {file_extension}")

        if file_extension in ['.mp4', '.avi', '.mov']:
            return 'video'
        elif file_extension in ['.mp3', '.wav', '.ogg', '.m4a']:
            return 'audio'
        elif file_extension == '.pdf':
            return 'pdf'
        elif file_extension == '.docx':
            return 'docx'
        elif file_extension == '.txt':
            return 'txt'

        # If extension check fails, try mime type
        mime_type, _ = mimetypes.guess_type(file_name)
        print(f"Debug: Detected MIME type is {mime_type}")

        if mime_type:
            if mime_type.startswith('video'):
                return 'video'
            elif mime_type.startswith('audio'):
                return 'audio'
            elif mime_type == 'application/pdf':
                return 'pdf'
            elif mime_type == 'application/vnd.openxmlformats-officedocument.wordprocessingml.document':
                return 'docx'
            elif mime_type == 'text/plain':
                return 'txt'

    return 'unknown'

In [ ]:
def process_pdf(file_path):
    print(f"Processing PDF file: {file_path}")
    text = ""
    with open(file_path, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        for page in reader.pages:
            text += page.extract_text() + "\n"

    summary = summarize_text(text)
    print("\nPDF Summary:")
    display(Markdown(summary))

def process_docx(file_path):
    print(f"Processing DOCX file: {file_path}")
    doc = Document(file_path)
    text = "\n".join([paragraph.text for paragraph in doc.paragraphs])

    summary = summarize_text(text)
    print("\nDOCX Summary:")
    display(Markdown(summary))

def process_txt(file_path):
    print(f"Processing TXT file: {file_path}")
    with open(file_path, 'r', encoding='utf-8') as file:
        text = file.read()

    summary = summarize_text(text)
    print("\nTXT Summary:")
    display(Markdown(summary))

In [ ]:
def process_youtube(url):
    try:
        audio_file = download_audio(url)
        if os.path.exists(audio_file):
            print(f"Debug: Audio file exists: {audio_file}")
            print(f"Debug: File size: {os.path.getsize(audio_file)} bytes")
            process_audio(audio_file)
        else:
            print(f"Error: Audio file {audio_file} not found after download")
    except Exception as e:
        print(f"An error occurred while processing YouTube URL: {str(e)}")


In [ ]:
def process_video(file_path):
    audio_file = 'audio.wav'
    if os.path.exists(audio_file):
        os.remove(audio_file)

    os.system(f"ffmpeg -i '{file_path}' -vn -acodec pcm_s16le -ar 44100 -ac 2 '{audio_file}' -y")
    process_audio(audio_file)

In [ ]:
from IPython.display import display, Markdown
def process_audio(file_path):
    print(f"Debug: Starting to process audio file: {file_path}")
    try:
        reader = VideoAudioReader()
        documents = reader.load_data(Path(file_path))
        transcript = documents[0].text
        print("Transcript:")
        print(transcript)

        summary = summarize_text(transcript)
        print("\nAudio/Video Summary:")
        #print(textwrap.fill(summary, width=80))
        display(Markdown(summary))
    except Exception as e:
        print(f"Error processing audio: {str(e)}")

In [ ]:
def summarize_text(text, max_tokens=500):
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "You are a helpful assistant that summarizes video transcripts."},
            {"role": "user", "content": f"Please summarize the following video transcript in about 250 words and be concise:\n\n{text}"}
        ],
        max_tokens=max_tokens
    )
    return response.choices[0].message.content

In [ ]:
def handle_upload(change):
    global processed_files
    if not change['new']:
        print("No file was uploaded.")
        return

    for filename, file_info in change['new'].items():
        if filename in processed_files:
            print(f"File {filename} has already been processed. Skipping.")
            continue

        print(f"Debug: Uploaded file info: {filename}")

        file_name = file_info['metadata']['name']
        content = file_info['content']

        print(f"Debug: File name is {file_name}")

        if not content:
            print("The uploaded file is empty.")
            continue

        with open(file_name, 'wb') as f:
            f.write(content)

        input_type = determine_input_type('', {'name': file_name})
        print(f"Debug: Determined input type is {input_type}")

        if input_type == 'video':
            process_video(file_name)
        elif input_type == 'audio':
            process_audio(file_name)
        elif input_type == 'pdf':
            process_pdf(file_name)
        elif input_type == 'docx':
            process_docx(file_name)
        elif input_type == 'txt':
            process_txt(file_name)
        else:
            print(f"Unsupported file type: {file_name}. Please upload a supported file type.")

        processed_files.add(filename)

In [ ]:
def reset_script(b):
    global processed_files

    # Clear all widgets
    url_input.value = ''
    upload_button.value.clear()
    upload_button._counter = 0

    # Delete all uploaded files
    for file in os.listdir():
        if file.endswith(('.mp4', '.mp3', '.wav', '.wav', '.pdf', '.docx', '.txt')):
            os.remove(file)

    # Clear the set of processed files
    processed_files.clear()

    # Remove any temporary directories created during processing
    if os.path.exists('temp'):
        shutil.rmtree('temp')

    # Clear output
    from IPython.display import clear_output
    clear_output()

    print("The script has been reset to a clean state.")
    print("All uploaded files have been deleted and the processed files list has been cleared.")
    print("Ready for new input. Please enter a YouTube URL or upload new files.")

    # Re-display widgets
    display(url_input)
    display(upload_button)
    display(process_button)
    display(reset_button)

In [ ]:
# Widgets
url_input = widgets.Text(
    value='',
    placeholder='Enter YouTube video URL',
    description='YouTube URL:',
    disabled=False
)

In [ ]:
upload_button = widgets.FileUpload(
    accept=".mp4,.mp3,.wav,.pdf,.docx,.txt",
    multiple=True
)

In [ ]:
process_button = widgets.Button(
    description='Process Media',
    disabled=False,
    button_style='',
    tooltip='Click to process the media'
)

In [ ]:
reset_button = widgets.Button(
    description='Reset',
    disabled=False,
    button_style='warning',
    tooltip='Click to reset the script to a clean state'
)

In [ ]:
def on_button_click(b):
    global processed_files
    url = url_input.value
    uploaded_files = upload_button.value

    if not url and not uploaded_files:
        print("Please enter a YouTube video URL or upload a video (MP4) or audio (MP3/WAV) file.")
        return

    if url:
        process_youtube(url)
    elif uploaded_files:
        for filename, file_info in uploaded_files.items():
            if filename in processed_files:
                print(f"File {filename} has already been processed. Skipping.")
                continue

            file_name = file_info['metadata']['name']
            input_type = determine_input_type('', {'name': file_name})

            try:
                if input_type == 'video':
                    process_video(file_name)
                elif input_type == 'audio':
                    process_audio(file_name)
                else:
                    print(f"Unsupported file type: {file_name}. Please upload an MP4 video or MP3/WAV audio file.")

                processed_files.add(filename)
            except Exception as e:
                print(f"An error occurred while processing {file_name}: {str(e)}")
    else:
        print("Unsupported input type. Please enter a YouTube video URL or upload a video (MP4) or audio (MP3/WAV) file.")

    # Clear the widgets
    url_input.value = ''
    upload_button.value.clear()
    upload_button._counter = 0

# Link button clicks to functions
process_button.on_click(on_button_click)
reset_button.on_click(reset_script)

# Display widgets
display(url_input)
display(upload_button)
display(process_button)
display(reset_button)

# Observe file upload
upload_button.observe(handle_upload, names='value')

# video_url:  "https://youtu.be/IxbR0yTMMY8?si=c5oTph_rRV2Sk_Gq"

Text(value='', description='YouTube URL:', placeholder='Enter YouTube video URL')

FileUpload(value={}, accept='.mp4,.mp3,.wav,.pdf,.docx,.txt', description='Upload', multiple=True)

Button(description='Process Media', style=ButtonStyle(), tooltip='Click to process the media')

Button(button_style='warning', description='Reset', style=ButtonStyle(), tooltip='Click to reset the script to…